Aqui, começaremos a atacar o problema de sincronizar nossos arquivos de audio com os acordes escritos. Primeiramente, tentaremos, para uma dada música, determinar os acordes por compasso. Faremos isso tomando como base os arquivos em formato xml.

In [1]:
import pandas as pd
import os
import xml.etree.ElementTree as ET
tree = ET.parse('corpus/xml/Ballad.xml')
root = tree.getroot()

In [14]:
part = root.findall('part')[0]
measures = part.findall('measure')
for measure in measures:
    chords = measure.findall('harmony')
    for chord in chords:
        try:
            note = chord.find('root/root-step').text

        except AttributeError: # se não achar a nota "root" do acorde, não tem acorde
            print('Measure: {}'.format(measure.attrib['number']), ' | ', 'No chords')
            continue
            
        if note:
            # alter é a alteração de um acorde (nem sempre existe): b ou #.
            alter = chord.find('root/root-alter')#.text
            
            # kind é o tipo do acorde (nem sempre existe): pode ser com 7a, 9a, 13a, por aí vai...
            kind = chord.find('kind')#.attrib['text']
            
            # bass é o baixo do acorde, quando diferente da tônica. Também nem sempre existe.
            bass = chord.find('bass/bass-step')#.text
            
            # bass_alter é e alteração do baixo, podendo ser b ou #. Também nem sempre existe.
            bass_alter = chord.find('bass/bass-alter')#.text
            
            attribs_dict = {'alter':alter, 
                            'kind':kind, 
                            'bass':bass, 
                            'bass_alter':bass_alter}

            for key, value in attribs_dict.items():
                if value is None:
                    attribs_dict[key] = ''
                elif key == 'kind':
                    attribs_dict[key] = value.attrib['text']
                else:
                    attribs_dict[key] = value.text
                    
            print('Measure: {}'.format(measure.attrib['number']), '|', 'Chord: ', note, 
                  attribs_dict['alter'], attribs_dict['kind'],'/',attribs_dict['bass'],attribs_dict['bass_alter'])
            

Measure: 3 | Chord:  E -1 min7 /  
Measure: 3 | Chord:  D -1  /  
Measure: 4 | Chord:  D -1  /  
Measure: 4 | Chord:  A -1  /  
Measure: 5 | Chord:  E -1 min7 /  
Measure: 5 | Chord:  D -1  / E -1
Measure: 6 | Chord:  D -1  / E -1
Measure: 6 | Chord:  A -1  / E -1
Measure: 7 | Chord:  E -1 min7 /  
Measure: 7 | Chord:  D -1  /  
Measure: 8 | Chord:  B   / E -1
Measure: 8 | Chord:  D -1 7 /  
Measure: 9 | Chord:  G -1 min7 /  
Measure: 10 | Chord:  E  min7 /  
Measure: 10 | Chord:  F 1  /  
Measure: 11 | Chord:  B  min7 /  
Measure: 12 | Chord:  G -1  /  
Measure: 12 | Chord:  B  7 /  
Measure: 13 | Chord:  E  maj7 /  
Measure: 14 | Chord:  A -1  /  
Measure: 14 | Chord:  A -1  / B -1
Measure: 15 | Chord:  E -1 min7 /  
Measure: 15 | Chord:  D -1  /  
Measure: 16 | Chord:  F  min7 /  
Measure: 16 | Chord:  B -1 7 /  
Measure: 17 | Chord:  G  min7 /  
Measure: 18 | Chord:  A -1 min7 /  
Measure: 18 | Chord:  A -1  /  
Measure: 19 | Chord:  G -1 7 /  
Measure: 20 | Chord:  A -1  / B -1
Me

Comparando com a partitura aberta no musescore parece fazer sentido, só preciso conseguir fazer com que mostre também os compassos em que não tem acorde (no caso da música que estamos usando como exemplo, os compassos 1 e 2).

Agora, vamos modificar ligeiramente o código acima para adicionar as informações em um dataframe.

In [38]:
df_teste = pd.DataFrame(columns=['Ballad'])
df_add = pd.DataFrame(columns=['Ballad'], 
                      index=[3], 
                      data='E -1 min7')

df_add_2 = pd.DataFrame(columns=['Ballad'], 
                      index=[3], 
                      data='D -1')

In [39]:
df_teste = df_teste.append(df_add)

In [40]:
df_teste.append(df_add_2)

,Ballad
3,E -1 min7
3,D -1


In [55]:
song = 'Ballad'
df_harmony = pd.DataFrame(columns=[song])

part = root.findall('part')[0]
measures = part.findall('measure')

for measure in measures:
    chords = measure.findall('harmony')
    for chord in chords:
        try:
            note = chord.find('root/root-step').text

        except AttributeError: # se não achar a nota "root" do acorde, não tem acorde
            print('Measure: {}'.format(measure.attrib['number']), ' | ', 'No chords')
            continue
            
        if note:
            # alter é a alteração de um acorde (nem sempre existe): b ou #.
            alter = chord.find('root/root-alter')#.text
            
            # kind é o tipo do acorde (nem sempre existe): pode ser com 7a, 9a, 13a, por aí vai...
            kind = chord.find('kind')#.attrib['text']
            
            # bass é o baixo do acorde, quando diferente da tônica. Também nem sempre existe.
            bass = chord.find('bass/bass-step')#.text
            
            # bass_alter é e alteração do baixo, podendo ser b ou #. Também nem sempre existe.
            bass_alter = chord.find('bass/bass-alter')#.text
            
            attribs_dict = {'alter':alter, 
                            'kind':kind, 
                            'bass':bass, 
                            'bass_alter':bass_alter}

            for key, value in attribs_dict.items():
                if value is None:
                    attribs_dict[key] = ''
                elif key == 'kind':
                    attribs_dict[key] = value.attrib['text']
                else:
                    attribs_dict[key] = value.text
                    
                    
            measure_number = measure.attrib['number']
            chord = note + attribs_dict['alter'] + attribs_dict['kind']
            chord_bass = attribs_dict['bass']+attribs_dict['bass_alter']
            complete_chord = chord + '/' + chord_bass
            
            if complete_chord.endswith('/'):
                complete_chord = complete_chord[:-1]
            
            df_chord = pd.DataFrame(columns=[song], index=[measure_number], data=complete_chord)
            df_harmony = df_harmony.append(df_chord)
            
df_harmony.index.names = ['Measure']

In [56]:
df_harmony

,Ballad
Measure,
3,E-1min7
3,D-1
4,D-1
4,A-1
5,E-1min7
5,D-1/E-1
6,D-1/E-1
6,A-1/E-1
7,E-1min7


Agora vamos procurar os momentos de mudança de acorde. Vasculhando os arquivos xml que temos em nosso corpus, pudemos ver que as informações sobre andamento (em bpm) e tipo de compasso encontram-se na seção do primeiro compasso. Segue:

#### Tipo de compasso

In [70]:
first_measure = measures[0]
cima = first_measure.find('attributes').find('time/beats').text
baixo = first_measure.find('attributes').find('time/beat-type').text

compasso = cima + '/' + baixo
compasso

'4/4'

#### Andamento

In [73]:
andamento = first_measure.find('sound').attrib['tempo'] + ' bpm'
andamento

'120 bpm'

Iterando sobre todos os arquivos do nosso corpus pra checar os compassos:

In [4]:
files = os.listdir('corpus/xml/')
for file in files:
    tree = ET.parse('corpus/xml/'+file)
    root = tree.getroot()

    part = root.findall('part')[0]
    measures = part.findall('measure')
    first_measure = measures[0]
    cima = first_measure.find('attributes').find('time/beats').text
    baixo = first_measure.find('attributes').find('time/beat-type').text

    compasso = cima + '/' + baixo
    
    print(file, compasso)

Sham Time.xml 4/4
AllOfMe_C.musicxml.1 4/4
BeninCity.musicxml.1 4/4
Mr.Broadway.xml 3/4
630blues.xml 4/4
WaltzInA,A.xml 12/8
SkateU.musicxml.1 4/4
TheOddCouple.xml 4/4
JumpinWithSymphonySid.xml 4/4
BeiMir.xml 4/4
CrossRiver.xml 4/4
James Bond.xml 4/4
MessAround.xml 4/4
Blues A La Mode.xml 4/4
LetItFlow.xml 4/4
Atchafalaya.xml 4/4
AtchFalaya.xml 4/4
MrPC.musicxml.1 4/4
This Here.xml 3/4
AloneIntheMorning.xml 4/4
ThatIsWhyYouAreOverweight.xml 4/4
HomeFires.xml 4/4
Chill.xml 4/4
WaynesThang.xml 4/4
Afrodisia.xml 4/4
stlouisblues.xml 4/4
FullHouse.musicxml.1 3/4
Number9.musicxml 4/4
ItCouldHappenToYou.musicxml 4/4
Ambidextrous.xml 4/4
Tricotism.xml 4/4
LuluSwing.musicxml.1 4/4
batida.xml 4/4
VivaDeFunk.xml 4/4
Blues for PK.xml 4/4
JoeAvery'sBlues.xml 4/4
ColdDuckTimeII.xml 4/4
PIWYWIPT.xml 4/4
Liberia.xml 4/4
SourceThe.xml 4/4
GrooveMerchant.xml 4/4
Ease Back.xml 4/4
Yeah You Right.xml 4/4
CrossRiver.xml.1 4/4
1612.musicxml 4/4
dorado 3.xml 4/4
ToTheTop.musicxml 4/4
dat dere.xml 4/4
AllOfM

A Little Busy.xml 4/4
Agogo.xml 4/4
Night Train ASax Transcription.xml 4/4
GreasyGBass.xml 4/4
RainbowVisions.xml 3/4
Spain.xml 4/4
herzog.xml 4/4
NewDeal.xml 4/4
Toad's Place.xml 4/4
Hattie Wall - 006 Drum Set.xml 4/4
chicken.xml 4/4
BirksWorks.musicxml.1 4/4
Play It Back.xml 4/4
TakeTheATrain.musicxml 4/4
MrPC.musicxml 4/4
Crisis.xml 4/4
TakeTheATrain.musicxml.1 4/4
Sequel, The.xml 4/4
bagsgroove 2.xml 4/4
ThereWillNeverBeAnotherYou.musicxml 4/4
Ornithology.xml 4/4
Superstition.xml 4/4
Mustang.musicxml 4/4
Whilemyladysleeps.xml 4/4
SirDuke.musicxml.1 4/4
Stutter.musicxml 4/4
Sandu.xml 4/4
WayBackHome.xml 4/4


ParseError: not well-formed (invalid token): line 1, column 6 (<string>)